<a href="https://colab.research.google.com/github/ggvaldez/LGPD-Script/blob/main/LGPD_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para cada trecho de código, siga as instruções e então rode o código

-------------

Defina aqui a URL, Secretaria e data da planilha a ser utilizada para a criação dos RIPDs

Esse código inicializa algumas informações.

In [12]:
url_google_sheets = 'https://docs.google.com/spreadsheets/d/DOCUMENTO_AQUI' # Insira a url dentro das aspas
secretaria = 'Secretaria de Inovação, Ciência e Tecnologia'
secretaria_curto = 'SICT'
data_doc = '25/07/2021'
data_long = '25 de Julho de 2021'

Autentica na sua conta Google e acessa a Planilha

Após rodar, autentique no google e copie e cole o código.


In [13]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url(url_google_sheets)
sheet = wb.worksheet('2-Inventario')
values_list = sheet.col_values(3)


Identifica os Inventários e importa outras planilhas

Exibe Planilhas faltantes (que constam em 1-Inventario, mas não existem no documento)



In [14]:
idp_sheets = {}
errors = []
in_header = True
for item in values_list:
  if (not(in_header)):
    try:
      idp_sheets[item] = wb.worksheet(item).get_all_values()
    except:
      errors.append("Falta a Planilha: " + item)
  elif (item == 'Nº Ref / ID'):
    in_header = False

print (errors)

['Falta a Planilha: DAD-01', 'Falta a Planilha: DAD-02']


Instala no ambiente uma biblioteca que manipula arquivos docx

In [ ]:
!pip install docxtpl
from docxtpl import DocxTemplate

Antes de executar, carregue (á esquerda, em arquivos, icone da pasta) a versão mais atualizada do arquivo 'template.docx' disponível em
https://github.com/ggvaldez/LGPD-Script/blob/main/template.docx

Caso queira fazer alterações em todos os documentos, modifique esse arquivo Word antes de enviar.

Gera todos os esboços RIPD

In [ ]:
for atvd_id in idp_sheets.keys():
  index = 0
  sheet_dict = {}
  table_riscos = []
  sheet_context = {}
  risco_id = 1
  for item in idp_sheets[atvd_id]:

    substring = item[0].split(" ",3)[0] # id da linha da planilha
    sheet_dict[substring] = index # pode ser removido em versão futura ?
    sheet_context["i" + substring.replace('.','i')] = idp_sheets[atvd_id][sheet_dict.get(substring)][1]
    
    if (substring.startswith('16.')): # tabela riscos
      if(idp_sheets[atvd_id][sheet_dict.get(substring)][5] != '' and idp_sheets[atvd_id][sheet_dict.get(substring)][5] != 'Não se aplica'):
        table_riscos.append({'id': risco_id,
                            'risco': idp_sheets[atvd_id][sheet_dict.get(substring)][1],
                            'impacto': idp_sheets[atvd_id][sheet_dict.get(substring)][5],
                            'prob': idp_sheets[atvd_id][sheet_dict.get(substring)][6]
                            })
        risco_id = risco_id + 1
    stri = ''
    it = item[0].split(" ")
    for s in range(2, len(it)):
        if (s == len(it) - 1):
          stri = stri + it[s]
        else:
          stri = stri + it[s] + ' '
    # indexa as colunas para facilitar o acesso ao dados no template
    sheet_context["i" + substring.replace('.','i') + 'a'] = stri 
    sheet_context["i" + substring.replace('.','i') + 'c'] = idp_sheets[atvd_id][sheet_dict.get(substring)][2]
    sheet_context["i" + substring.replace('.','i') + 'd'] = idp_sheets[atvd_id][sheet_dict.get(substring)][3]
    sheet_context["i" + substring.replace('.','i') + 'e'] = idp_sheets[atvd_id][sheet_dict.get(substring)][4]
    sheet_context["i" + substring.replace('.','i') + 'f'] = idp_sheets[atvd_id][sheet_dict.get(substring)][5]
    sheet_context["i" + substring.replace('.','i') + 'g'] = idp_sheets[atvd_id][sheet_dict.get(substring)][6]
    index = index + 1
  sheet_context['secretaria'] = secretaria
  sheet_context['secretariacurto'] = secretaria_curto
  sheet_context['datadoc'] = data_doc
  sheet_context['datalong'] = data_long
  remove_keys = []
  add_keys = []
  for i in sheet_context.keys(): # limpa celulas em branco para poder usar testes 'if' no template
    if sheet_context[i] == '':
      remove_keys.append(i)
  for i in remove_keys:
    del(sheet_context[i])
  for i in sheet_context.keys(): # adiciona chaves especificas para remover cabeçalhos desnecessários em "escopo do documento"
    for j in range (1, 15):
      if (i == 'i8i'+str(j)):
        add_keys.append('i8existe')
      for k in range (1, 15):
        if (i == 'i7i'+str(j)+'i'+str(k)):
          add_keys.append('i7i' + str(j) + 'existe')
  for i in add_keys:
    sheet_context[i] = 'S'
  sheet_context['table_riscos'] = table_riscos
  doc = DocxTemplate("template.docx")
  doc.render(sheet_context)
  doc.save("RIPD "+atvd_id +" esboço.docx")
  print ("Gerado arquivo RIPD "+atvd_id +" esboço.docx")


Zipa todos os arquivos .docx para facilitar o download

In [ ]:
!zip -r ./RIPD-esboco.zip ./*.docx
